### part (a)

Using Krichoff's current law, the currents through junction 1, 2 and 3 can be written as:
\begin{equation}
    I_{t1} = I_{12} + I_{13} \\
    I_{t2} + I_{12} = I_{23} + I_{2g} \\
    I_{13} + I_{23} = I_{3g}
\end{equation}

where $t$ and $g$ refer to the positive terminal and the ground resp.

Multiplying these by $R$, we get:
\begin{equation}
    V_{+} - V_{1} = (V_{1} - V_{2}) + (V_{1}-V_{3}) \\
    (V_{+} - V_{2}) + (V_{1} - V_{2}) = (V_{2} - V_{3}) + (V_{2} - 0) \\
    (V_{1} - V_{3}) + (V_{2} - V_{3}) = V_{3} - 0
\end{equation}

Re-arranging the terms, this can be written as

$$
\begin{bmatrix}
    3 & -1 & -1 \\
    -1 & 4 & -1 \\
    -1 & -1 & 3
\end{bmatrix}
\begin{bmatrix}
    V_{1}\\
    V_{2}\\
    V_{3}
\end{bmatrix}
=
\begin{bmatrix}
    V_{+}\\
    V_{+}\\
    0
\end{bmatrix}
$$

The LU decomposition can be written as:

$$
\begin{bmatrix}
    3 & -1 & -1 \\
    -1 & 4 & -1 \\
    -1 & -1 & 3
\end{bmatrix}
=
\begin{bmatrix}
    1 & 0 & 0 \\
    -\frac{1}{3} & 1 & 0 \\
    -\frac{1}{3} & -\frac{13}{3} & 1
\end{bmatrix}
\begin{bmatrix}
    3 & -1 & -1 \\
    0 & \frac{11}{3} & -\frac{4}{3} \\
    0 & 0 & \frac{24}{11}
\end{bmatrix}
$$

Using Gaussian elimination, the solution is given by:
$$
\begin{bmatrix}
    V_{1}\\
    V_{2}\\
    V_{3}
\end{bmatrix}
=
\begin{bmatrix}
    \frac{25}{8}\\
    \frac{5}{2}\\
    \frac{15}{8}
\end{bmatrix}
$$

In [28]:
import numpy as np

In [29]:
np.linalg.solve([[3,-1,-1],[-1,4,-1],[-1,-1,3]], [5, 5, 0])

array([3.125, 2.5  , 1.875])

### part (b)

For N internal junctions, all except the first two and the last two junctions are 4-way junctions. They have 2 currents going in, and 2 coming out. 
That is, for the $i^{th}$ junction (excluding first two and last two), Krichoff's law gives:
$$
I_{i,i-1} + I_{i,i-2} = I_{i+2,i} + I_{i+1,i} 
$$
Multiplying by $R$, we get:
$$
(V_{i} - V_{i-1}) + (V_{i} - V_{i-2}) = (V_{i+2} - V_{i}) + (V_{i+1} - V_{i})
$$

This equation can be rearranged to give:
$$- V_{i-1} - V_{i-2} + 4V_{i} -V_{i+1} -V_{i+2} = 0 $$

Junctions $1$, $2$, $N-1$ and $N$ are connected to the terminals, so for them the equations are slightly different.

These equations can be written as:

$$
\begin{bmatrix}
    3 & -1 & -1 &  &  &  & \\
    -1 & 4 & -1 & -1 &  &  &  \\
    -1 & -1 & 4 & -1 & -1 &  & & ...\\
     & -1 & -1 & 4 & -1 & -1 &  \\
     &  & -1 & -1 & 4 & -1 & -1 \\
     & & & . & & & \\
     & & & . & & & \\
     & & & . & & & \\
     & & & & & & & & -1 & -1 & 4 & -1\\
     & & & & & & & & & -1 & -1 & 3 
\end{bmatrix}
\begin{bmatrix}
    V_{1}\\
    V_{2}\\
    V_{3}\\
    .\\
    .\\
    .\\
    V_{N-1}\\
    V_{N}
\end{bmatrix}
=
\begin{bmatrix}
    V_{+}\\
    V_{+}\\
    0\\
    0\\
    .\\
    .\\
    .\\
\end{bmatrix}
$$


### part (c)

In [34]:
A = [[3,-1,-1,0,0,0],[-1,4,-1,-1,0,0],[-1,-1,4,-1,-1,0],[0,-1,-1,4,-1,-1],[0,0,-1,-1,4,-1],[0,0,0,-1,-1,3]]
b = [5,5]+[0]*4
print(np.linalg.solve(A, b))

[3.7254902  3.43137255 2.74509804 2.25490196 1.56862745 1.2745098 ]


### part (d)

In [43]:
######################################################################
#
# Function to solve a banded system of linear equations using
# Gaussian elimination and backsubstitution
#
# x = banded(A,v,up,down)
#
# This function returns the vector solution x of the equation A.x = v,
# where v is an array representing a vector of N elements, either real
# or complex, and A is an N by N banded matrix with "up" nonzero
# elements above the diagonal and "down" nonzero elements below the
# diagonal.  The matrix is specified as a two-dimensional array of
# (1+up+down) by N elements with the diagonals of the original matrix
# along its rows, thus:
#
#   (  -   -  A02 A13 A24 ...
#   (  -  A01 A12 A23 A34 ...
#   ( A00 A11 A22 A33 A44 ...
#   ( A10 A21 A32 A43 A54 ...
#   ( A20 A31 A42 A53 A64 ...
#
# Elements represented by dashes are ignored -- it doesn't matter what
# these elements contain.  The size of the system is taken from the
# size of the vector v.  If the matrix A is larger than NxN then the
# extras are ignored.  If it is smaller, the program will produce an
# error.
#
# The function is compatible with version 2 and version 3 of Python.
#
# Written by Mark Newman <mejn@umich.edu>, September 4, 2011
# You may use, share, or modify this file freely
#
######################################################################

from numpy import copy

def banded(Aa,va,up,down):

    # Copy the inputs and determine the size of the system
    A = copy(Aa)
    v = copy(va)
    N = len(v)

    # Gaussian elimination
    for m in range(N):

        # Normalization factor
        div = A[up,m]

        # Update the vector first
        v[m] /= div
        for k in range(1,down+1):
            if m+k<N:
                v[m+k] -= A[up+k,m]*v[m]

        # Now normalize the pivot row of A and subtract from lower ones
        for i in range(up):
            j = m + up - i
            if j<N:
                A[i,j] /= div
                for k in range(1,down+1):
                    A[i+k,j] -= A[up+k,m]*A[i,j]

    # Backsubstitution
    for m in range(N-2,-1,-1):
        for i in range(up):
            j = m + up - i
            if j<N:
                v[m] -= A[i,j]*v[j]

    return v

In [42]:
N = 10000
 
diag = 4*np.ones(N)
diag[0], diag[-1] = 3, 3

v = np.zeros(N)
v[0], v[1] = 5, 5
A_banded = -1*np.ones(shape=(5,N))
A_banded[2,:] = diag 

x = banded(A_banded, v, 2, 2)
print(x)

[4.99888228e+00 4.99861842e+00 4.99802841e+00 ... 1.97158611e-03
 1.38158071e-03 1.11772227e-03]
